# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [118]:
import os
if 'A306709' in os.environ['USERNAME']:
    print("Running on Christophs computer: update proxy settings.")
    os.environ["http_proxy"] = "http://sia-lb.telekom.de:8080"
    os.environ["https_proxy"] = "http://sia-lb.telekom.de:8080"
else:
    print("Running on any computer but not Christophs: don't update any proxy settings.")


Running on Christophs computer: update proxy settings.


In [119]:
from datasets import load_dataset

splits = ["train", "test"]
# I had a lot of trouble using the emotion dataset: I had a bad initial accurance and very bad learning rates, so I switched to anohter dataset.
#ds = {split: ds for split, ds in zip(splits, load_dataset("dair-ai/emotion", split=splits))}
ds = {split: ds for split, ds in zip(splits, load_dataset("SetFit/bbc-news", split=splits))}

for split in splits:
    ds[split] = ds[split].shuffle(seed=37).select(range(500))

print(ds)
print(ds['train'][0])
print(ds['test'][0])


{'train': Dataset({
    features: ['text', 'label', 'label_text'],
    num_rows: 500
}), 'test': Dataset({
    features: ['text', 'label', 'label_text'],
    num_rows: 500
})}
{'text': 'jobs go at oracle after takeover oracle has announced it is cutting about 5 000 jobs following the completion of its $10.3bn takeover of its smaller rival peoplesoft last week.  the company said it would retain more than 90% of peoplesoft product development and product support staff. the cuts will affect about 9% of the 55 000 staff of the combined companies. oracle s 18-month fight to acquire peoplesoft was one of the most drawn-out and hard-fought us takeover battles of recent times. the merged companies are set to be a major force in the enterprise software market  second only in size to germany s sap.  in a statement  oracle said it began notifying staff of redundancies on friday and the process would continue over the next 10 days.  by retaining the vast majority of peoplesoft technical staff  ora

In [120]:
# Let's have a look into the labels of the dataset:
labels_done = []
for a_test in ds['test']:
    if not a_test["label"] in labels_done:
        labels_done.insert(0, a_test["label"])
        print(f'{a_test["label"]}: {a_test["label_text"]}')


3: entertainment
0: tech
4: politics
2: sport
1: business


In [121]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased") # distilbert-base-uncased, gpt2

def preprocess_function(examples):
    """Preprocess the imdb dataset by returning tokenized examples."""
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_ds = {}
for split in splits:
    tokenized_ds[split] = ds[split].map(preprocess_function, batched=True)

print(tokenized_ds)
print(tokenized_ds['train']['input_ids'][0][:20])

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

{'train': Dataset({
    features: ['text', 'label', 'label_text', 'input_ids', 'attention_mask'],
    num_rows: 500
}), 'test': Dataset({
    features: ['text', 'label', 'label_text', 'input_ids', 'attention_mask'],
    num_rows: 500
})}
[101, 5841, 2175, 2012, 14721, 2044, 15336, 14721, 2038, 2623, 2009, 2003, 6276, 2055, 1019, 2199, 5841, 2206, 1996, 6503]


In [122]:
from transformers import AutoModelForSequenceClassification

#id2label_emotion={0: "sadness", 1: "joy", 2: "love", 3: "anger", 4: "fear", 5: "surprise"}
#label2id_emotion={"sadness": 0, "joy": 1, "love": 2, "anger": 3, "fear": 4, "surprise": 5}
id2label={0: "tech", 1: "business", 2: "sport", 3: "entertainment", 4: "politics"}
label2id={"tech": 0, "business": 1, "sport": 2, "entertainment": 3, "politics": 4}

def load_base_model():
    return AutoModelForSequenceClassification.from_pretrained(
        "distilbert-base-uncased",  # gpt2?
        num_labels=len(id2label),
        id2label=id2label,
        label2id=label2id,
    )

model = load_base_model()

# Freeze all the parameters of the base model
# Hint: Check the documentation at https://huggingface.co/transformers/v4.2.2/training.html
#for param in model.base_model.parameters():
#    param.requires_grad = False

print(model)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [123]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments

def compute_metrics(eval_pred):
    print(eval_pred)
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

# this Trainer object is only used to evaluate the initial accuracy, not for training:
# maybe I can skip some of the parameters?
def evaluate_model(a_model, a_tokenized_data, a_tokenizer):
    local_trainer = Trainer(
        model=a_model,
        args=TrainingArguments(
            #output_dir="./data/dummy",
            learning_rate=2e-5,
            per_device_train_batch_size=1,
            per_device_eval_batch_size=1,
            evaluation_strategy="epoch",
            save_strategy="epoch",
            label_names=["labels"],
            num_train_epochs=1,
            weight_decay=0.01,
            load_best_model_at_end=True,
        ),
        train_dataset=a_tokenized_data["train"],
        eval_dataset=a_tokenized_data["test"],
        tokenizer=a_tokenizer,
        data_collator=DataCollatorWithPadding(tokenizer=a_tokenizer),
        compute_metrics=compute_metrics,
    )
    return local_trainer.evaluate()


initial_performance = evaluate_model(model, tokenized_ds, tokenizer)

def print_accuracy_values(a_performance, a_description, print_performance_object=True):
    local_accuracy = a_performance['eval_accuracy']
    if print_performance_object:
        print(a_performance)
    print('*' * 34)
    print(f'***** {a_description}: {local_accuracy} *****')
    print('*' * 34)

print_accuracy_values(initial_performance, "Initial accuracy")


C:\Users\A306709\dev\2025-02-AIWithPython\NanoDegreeCodeSammlung\.venv\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\A306709\AppData\Local\Temp\ipykernel_21412\4026100038.py:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  local_trainer = Trainer(


{'eval_loss': 1.6115407943725586, 'eval_model_preparation_time': 0.001, 'eval_accuracy': 0.27, 'eval_runtime': 93.3976, 'eval_samples_per_second': 5.353, 'eval_steps_per_second': 5.353}
**********************************
***** Initial accuracy: 0.27 *****
**********************************


In [147]:
import torch

def check_single_message(a_model, a_text, a_tokenizer):
    local_inputs = a_tokenizer(a_text, return_tensors="pt")
    tensors = a_model(**local_inputs).logits
    print(tensors)
    predicted_label_id = torch.argmax(tensors, dim=-1).item()
    return id2label[predicted_label_id]

# Play around to see how well the model works:
print(type(model))
print(check_single_message(model, "Bayern Munich won the latest football match.", tokenizer))
print(check_single_message(model, "The new insurance company has millions of customers and earns billions of dollars.", tokenizer))
print(check_single_message(model, "There is a new game show on television which entertains thousands of people.", tokenizer))
print(check_single_message(model, "Nvidia developed the next generation of ai chips.", tokenizer))


<class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'>
tensor([[ 0.0006,  0.0131,  0.1014,  0.0041, -0.0399]],
       grad_fn=<AddmmBackward0>)
sport
tensor([[ 0.0388,  0.1038,  0.0202, -0.0643, -0.0385]],
       grad_fn=<AddmmBackward0>)
business
tensor([[-0.0271,  0.0067,  0.0093,  0.0271, -0.0069]],
       grad_fn=<AddmmBackward0>)
entertainment
tensor([[ 0.0910,  0.0747,  0.0319, -0.0072, -0.0623]],
       grad_fn=<AddmmBackward0>)
tech


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [148]:
# see: https://huggingface.co/docs/peft/main/en/conceptual_guides/lora
from peft import LoraConfig, get_peft_model

# not sure what these parameters exactly stand for. Look at this later again.
config = LoraConfig(task_type="SEQ_CLS", target_modules=["q_lin", "k_lin", "v_lin", "out_lin"])
lora_model = get_peft_model(model, config)
# wtf is that? our coach gave us this line:
lora_model.config.pad_token_id = lora_model.config.eos_token_id
lora_model.print_trainable_parameters()

# Don't(!!) do this, because we want exactly NOT to change all params but only the new ones:
#for param in lora_model.parameters():
#    param.requires_grad = True


trainable params: 889,349 || all params: 67,846,666 || trainable%: 1.3108


In [126]:
lora_trainer = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir="./data/emotion",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        label_names=["labels"],
        greater_is_better=True,
        num_train_epochs=5,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)


C:\Users\A306709\AppData\Local\Temp\ipykernel_21412\643945624.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  lora_trainer = Trainer(


In [127]:
lora_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.553425,0.698000
2,No log,1.506407,0.708000
3,No log,1.467077,0.764000
4,No log,1.441747,0.808000
5,No log,1.432822,0.808000


TrainOutput(global_step=160, training_loss=1.504368495941162, metrics={'train_runtime': 1750.1251, 'train_samples_per_second': 1.428, 'train_steps_per_second': 0.091, 'total_flos': 338016414720000.0, 'train_loss': 1.504368495941162, 'epoch': 5.0})

In [128]:
# useless check because this data is always printed in the section above:
#lora_eval = evaluate_model(lora_model, tokenized_ds, tokenizer)
#print_accuracy_values(lora_eval, "LORA model after training")

# save model and tokenizer:
lora_model.save_pretrained("trained-lora-model")
tokenizer.save_pretrained("trained-lora-model")


('trained-lora-model\\tokenizer_config.json',
 'trained-lora-model\\special_tokens_map.json',
 'trained-lora-model\\vocab.txt',
 'trained-lora-model\\added_tokens.json',
 'trained-lora-model\\tokenizer.json')

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [129]:
from peft import PeftModel

base_model = load_base_model()
loaded_lora_model = PeftModel.from_pretrained(base_model, "trained-lora-model")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [130]:
loaded_tokenizer = AutoTokenizer.from_pretrained("trained-lora-model")
# repeat the spooky code line from our trainer:
loaded_lora_model.config.pad_token_id = loaded_lora_model.config.eos_token_id

loaded_tokenized_ds = {}
for split in splits:
    loaded_tokenized_ds[split] = ds[split].map(preprocess_function, batched=True)

print(loaded_tokenized_ds)
print(loaded_tokenized_ds['train']['input_ids'][0][:20])


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

{'train': Dataset({
    features: ['text', 'label', 'label_text', 'input_ids', 'attention_mask'],
    num_rows: 500
}), 'test': Dataset({
    features: ['text', 'label', 'label_text', 'input_ids', 'attention_mask'],
    num_rows: 500
})}
[101, 5841, 2175, 2012, 14721, 2044, 15336, 14721, 2038, 2623, 2009, 2003, 6276, 2055, 1019, 2199, 5841, 2206, 1996, 6503]


In [131]:
loaded_lora_model.eval()

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): DistilBertForSequenceClassification(
      (distilbert): DistilBertModel(
        (embeddings): Embeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (transformer): Transformer(
          (layer): ModuleList(
            (0-5): 6 x TransformerBlock(
              (attention): DistilBertSdpaAttention(
                (dropout): Dropout(p=0.1, inplace=False)
                (q_lin): lora.Linear(
                  (base_layer): Linear(in_features=768, out_features=768, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Identity()
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=768, out_features=8, 

In [132]:
loaded_model_performance = evaluate_model(loaded_lora_model, loaded_tokenized_ds, loaded_tokenizer)

print_accuracy_values(loaded_model_performance, "Loaded model accuracy")
print_accuracy_values(initial_performance, "To compare: initial accuracy was", False)

C:\Users\A306709\dev\2025-02-AIWithPython\NanoDegreeCodeSammlung\.venv\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\A306709\AppData\Local\Temp\ipykernel_21412\4026100038.py:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  local_trainer = Trainer(


{'eval_loss': 1.553424596786499, 'eval_model_preparation_time': 0.003, 'eval_accuracy': 0.698, 'eval_runtime': 99.6052, 'eval_samples_per_second': 5.02, 'eval_steps_per_second': 5.02}
**********************************
***** Loaded model accuracy: 0.698 *****
**********************************
**********************************
***** To compare: initial accuracy was: 0.27 *****
**********************************


In [146]:
# Here I play around to see how well the loaded model works:
print(type(loaded_lora_model))
print(check_single_message(loaded_lora_model, "Bayern Munich won the latest football match.", loaded_tokenizer))
print(check_single_message(loaded_lora_model, "The new insurance company has millions of customers and earns billions of dollars.", loaded_tokenizer))
print(check_single_message(loaded_lora_model, "There is a new game show on television which entertains thousands of people.", loaded_tokenizer))
print(check_single_message(loaded_lora_model, "Nvidia developed the next generation of ai chips.", loaded_tokenizer))


<class 'peft.peft_model.PeftModelForSequenceClassification'>
tensor([[ 0.0006,  0.0131,  0.1014,  0.0041, -0.0399]],
       grad_fn=<AddmmBackward0>)
sport
tensor([[ 0.0388,  0.1038,  0.0202, -0.0643, -0.0385]],
       grad_fn=<AddmmBackward0>)
business
tensor([[-0.0271,  0.0067,  0.0093,  0.0271, -0.0069]],
       grad_fn=<AddmmBackward0>)
entertainment
tensor([[ 0.0910,  0.0747,  0.0319, -0.0072, -0.0623]],
       grad_fn=<AddmmBackward0>)
tech
